# Introduction

My project consists of an analysis of the European Social Survey, focusing on the modules which measure attitudes to immigration in Ireland. 

### Description of dataset

The European Social Survey (ESS) is a cross-national survey which aims to monitor public attitudes within Europe. It is conducted every two years, and it has been administered in over 40 countries since 2001. Each round of the survey is designed to capture a cross-sectional random sample in each of the participating countries.

For this project, round X was used. For ireland only??? decide later.

### Research Questions
DRAFT QUESTIONS - EDIT OR REMOVE LATER ONCE CLEANING IS DONE
1)	What was the level of (anti)- immigrant sentiment in Ireland in 2020?
2)	What was the relationship between an individual’s attitude to immigrants and their income level?
3)	What was the relationship between an individual’s attitude to immigrants and whether they had completed third-level education?
4)	What was the relationship between an individual’s attitude to immigrants and their choice of political party?


### Part 1: Reading in the data

In [2]:
#import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import scipy.stats # statistical functions
# plotting
#import matplotlib.pyplot as plt
#import seaborn as sns

In [3]:
# start with loading all available data into a data frame
df = pd.read_csv("./data.csv")
df.info()

/tmp/ipykernel_23455/3021999600.py:2: DtypeWarning: Columns (635,636,637,638,639,644,645,646,647,648,666,667,668,669,677,678,679,680,681,682,683,684,1534,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1606,1607,1608) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24250 entries, 0 to 24249
Columns: 2785 entries, name to recon
dtypes: float64(2615), int64(128), object(42)
memory usage: 515.3+ MB


### Part 2: Data Cleaning and Processing

In [7]:
# first see the fields and data types
df.dtypes

name         object
essround      int64
edition     float64
proddate     object
idno          int64
             ...   
respc19a    float64
symtc19     float64
symtnc19    float64
vacc19      float64
recon       float64
Length: 2785, dtype: object

In [6]:
df.head()

,name,essround,edition,proddate,idno,cntry,dweight,pspwght,pweight,anweight,...,testji5,testji6,testji7,testji8,testji9,respc19a,symtc19,symtnc19,vacc19,recon
0,ESS1e06_7,1,6.7,23.11.2023,101,IE,0.9165,0.733937,0.150276,0.110293,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ESS1e06_7,1,6.7,23.11.2023,102,IE,0.9165,0.700541,0.150276,0.105274,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ESS1e06_7,1,6.7,23.11.2023,104,IE,1.8331,1.445871,0.150276,0.217279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ESS1e06_7,1,6.7,23.11.2023,105,IE,0.9165,1.030524,0.150276,0.154863,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ESS1e06_7,1,6.7,23.11.2023,106,IE,1.3748,1.084384,0.150276,0.162957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are too many fields so let's cut it down to just what we want.

In [ ]:
# take the last five years of rounds for now by specify ess round 7 - 10. 
# see: https://pandas.pydata.org/docs/reference/api/pandas.Series.between.html 
df_v2 = df[df['essround'].between(7, 11)]

In [9]:
# Using the data dictionary, just take the columns that we are interested in
# administrative variables, socio-demographics, political allegiance and immigrant sentiment items 
df_v3 = df_v2[ ['name', 'edition', 'proddate', 'idno', 'anweight', 'edlvdie', 'isco08', 'hinctnta', 'prtvtdie', 'imsmetn', 'imdfetn', 'impcntr', 'imbgeco', 'imueclt', 'imwbcnt'] ]
print(type(df_v3))
df_v3

<class 'pandas.core.frame.DataFrame'>


,name,edition,proddate,idno,anweight,edlvdie,isco08,hinctnta,prtvtdie,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt
13100,ESS7e02_3,2.3,23.11.2023,2,0.074851,12.0,5223.0,3.0,NaN,2,2,2,5,8,8
13101,ESS7e02_3,2.3,23.11.2023,4,0.154009,9.0,4211.0,3.0,NaN,1,2,2,5,9,6
13102,ESS7e02_3,2.3,23.11.2023,5,0.341264,3.0,66666.0,1.0,NaN,2,2,2,5,5,6
13103,ESS7e02_3,2.3,23.11.2023,8,0.205346,2.0,9112.0,1.0,NaN,2,3,3,2,3,2
13104,ESS7e02_3,2.3,23.11.2023,11,0.099170,2.0,8331.0,3.0,NaN,2,3,3,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24245,ESS11e04,4.0,19.11.2025,86371,0.149981,15.0,2221.0,77.0,NaN,2,2,2,5,5,5
24246,ESS11e04,4.0,19.11.2025,86380,0.071264,13.0,66666.0,4.0,NaN,1,1,1,4,6,6
24247,ESS11e04,4.0,19.11.2025,86423,0.151995,14.0,1321.0,5.0,NaN,3,3,3,8,7,8
24248,ESS11e04,4.0,19.11.2025,86468,0.057569,3.0,66666.0,1.0,NaN,2,4,4,5,5,5
